In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
from pysurvival.models.multi_task import NeuralMultiTaskModel
from pysurvival.models.survival_forest import ConditionalSurvivalForestModel
from pysurvival.utils.display import compare_to_actual
import mlflow
import logging
import random

In [3]:
data = pd.read_csv('./loan_data.csv')

In [4]:
data.columns

Index(['Loan ID', 'Customer ID', 'Loan Status', 'Current Loan Amount', 'Term',
       'Credit Score', 'Annual Income', 'Years in current job',
       'Home Ownership', 'Purpose', 'Monthly Debt', 'Years of Credit History',
       'Months since last delinquent', 'Number of Open Accounts',
       'Number of Credit Problems', 'Current Credit Balance',
       'Maximum Open Credit', 'Bankruptcies', 'Tax Liens'],
      dtype='object')

In [5]:
cols = list(map(lambda x: x.lower().replace(" ", "_"), data.columns))

In [6]:
data.columns = cols

In [7]:
x = data.dropna(axis=0)

In [14]:
trial = x[['term', 'home_ownership', 'purpose']]

In [15]:
dv = DictVectorizer(sparse=False)

In [16]:
dv.fit_transform(trial.to_dict(orient="records")).shape

(36423, 22)

In [13]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36423 entries, 2 to 99997
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   loan_id                       36423 non-null  object 
 1   customer_id                   36423 non-null  object 
 2   loan_status                   36423 non-null  object 
 3   current_loan_amount           36423 non-null  float64
 4   term                          36423 non-null  object 
 5   credit_score                  36423 non-null  float64
 6   annual_income                 36423 non-null  float64
 7   years_in_current_job          36423 non-null  object 
 8   home_ownership                36423 non-null  object 
 9   purpose                       36423 non-null  object 
 10  monthly_debt                  36423 non-null  float64
 11  years_of_credit_history       36423 non-null  float64
 12  months_since_last_delinquent  36423 non-null  float64
 13  n